In [1]:
import pandas as pd

# Transit

In [734]:
transit_df = pd.read_excel(r'data\network_summary_detailed.xlsx', sheetname='Transit Summaries')
transit_df.index = transit_df['route_code']

In [735]:
# Reformat data
dict_result = {}
for field in ['board','time']:
    df = pd.DataFrame(transit_df[[tod+'_'+ field for tod in todlist]].stack())
    df.rename(columns={0:field}, inplace=True)
    df['tod'] = [i.split('_')[0] for i in df.index.get_level_values(1)]
    df['route_id'] = df.index.get_level_values(0)
    df.reset_index(inplace=True, drop=True)
    
    dict_result[field] = df



In [736]:
model = dict_result['board']
model.rename(columns={'board':'model_boardings'}, inplace=True)

In [737]:
# Load observed results
obs = pd.read_csv('data/transit_boardings_2014.csv')
obs.index = obs['PSRC_Rte_ID']

obs.drop([u'Unnamed: 0','PSRC_Rte_ID','SignRt'],axis=1,inplace=True)

# [i.split('_')[-1] for i in obs.columns]

# Rename columns
# obs[['hour_'+ str(i) for i in xrange(1,25)]].columns = [str(i) for i in xrange(1,25)]
df = pd.DataFrame(obs.stack())

df['hour'] = df.index.get_level_values(1)
df['route_id'] = df.index.get_level_values(0)

df.reset_index(inplace=True, drop=True)
df.rename(columns={0:'observed_boardings'}, inplace=True)

df['hour'] = df['hour'].apply(lambda row: row.split('_')[-1])

In [738]:
# look up psrc time of day 
tod_list = ['5to6','6to7','7to8','8to9','9to10','10to14','14to15','16to17','17to18','18to20','20to5']

tod_lookup = {  0:'20to5',
                1:'20to5',
                2:'20to5',
                3:'20to5',
                4:'20to5',
                5:'5to6',
                6:'6to7',
                7:'7to8',
                8:'8to9',
                9:'9to10',
                10:'10to14',
                11:'10to14',
                12:'10to14',
                13:'10to14',
                14:'14to15',
                15:'15to16',
                16:'16to17',
                17:'17to18',
                18:'18to20',
                19:'18to20',
                20:'18to20',
                21:'20to5',
                22:'20to5',
                23:'20to5' }

In [739]:
len(df)

9192

In [740]:
# Load tod lookup
tod_df = pd.DataFrame(data=tod_lookup.values(),index=tod_lookup.keys(), columns=['tod'])
tod_df['hour'] = tod_df.index.astype('str')

df = pd.merge(df,tod_df,on='hour')
df.drop('hour', axis=1,inplace=True)

In [741]:
len(df)

8809

In [742]:
# Group by tod
df = df.groupby(['tod','route_id']).sum()
df['tod'] = df.index.get_level_values(0)
df['route_id'] = df.index.get_level_values(1)
df.reset_index(inplace=True, drop=True)

obs = df

In [743]:
len(obs)

4584

In [744]:
len(model)

1955

In [750]:
# Merge model and observed data
df = pd.merge(obs,model,on=['tod','route_id'])
df.to_csv('transit_boarding.csv')
len(df)

1662

In [746]:
# Add geography
obs.groupby('tod').count()

,observed_boardings,route_id
tod,,
10to14,382,382
14to15,382,382
15to16,382,382
16to17,382,382
17to18,382,382
18to20,382,382
20to5,382,382
5to6,382,382
6to7,382,382


# Traffic Counts

In [317]:
counts_df = pd.read_excel(r'J:\Projects\Soundcast\network_summary_detailed.xlsx', sheetname='Counts Output')


In [318]:
# Reset indices
counts_df['loop_INode'] = counts_df.index.get_level_values(0)
counts_df['loop_JNode'] = counts_df.index.get_level_values(1)

counts_df.reset_index(inplace=True,drop=True)

# Drop first row - included column names as a row
counts_df.drop(0,inplace=True)

# drop extra columns
counts_df.drop([u'OBJECTID_1', u'Join_Count', u'TARGET_FID', u'OBJECTID', u'SR', u'RID', 
         u'MP', u'ARM', u'Type_', u'Lanes', u'Oneway', u'Dir', u'ID',
         u'HOV_I', u'HOV_J'],
       axis=1, inplace=True)

- observed

In [391]:
df = counts_df

for i in xrange(24):
    if i < 10:
        df = df.rename(columns={'Vol_0'+str(i): str(i)})
    df = df.rename(columns={'Vol_'+str(i): str(i)})
    
df = df[[str(i) for i in xrange(24)]+['NewINode']]
df = df.set_index(keys='NewINode',drop=True)

# realign
df = pd.DataFrame(df.stack())

df['hour'] = df.index.get_level_values(1)
df['NewINode'] = df.index.get_level_values(0)
df.rename(columns={0:'observed'}, inplace=True)
df.reset_index(inplace=True,drop=True)

# Aggregate results by soundcast tod dict

# Load tod lookup
tod_df = pd.DataFrame(data=tod_lookup.values(),index=tod_lookup.keys(), columns=['tod'])
tod_df['hour'] = tod_df.index.astype('str')

df = pd.merge(df,tod_df,on='hour')
df.drop('hour', axis=1,inplace=True)

# Group by tod
df = df.groupby(['tod','NewINode']).sum()
df['tod'] = df.index.get_level_values(0)
df['NewINode'] = df.index.get_level_values(1)
df.reset_index(inplace=True, drop=True)

obs = df

In [394]:
len(obs)

2136

In [395]:
len(mod)

2231

- model

In [397]:
df = counts_df


df = df[['vol'+str(i) for i in tod_list]+['NewINode']]
df = df.set_index(keys='NewINode',drop=True)

# realign
df = pd.DataFrame(df.stack())

df['tod'] = df.index.get_level_values(1)
df['tod'] = df['tod'].apply(lambda row: row.split('vol')[-1])
df['NewINode'] = df.index.get_level_values(0)
df.rename(columns={0:'model'}, inplace=True)
df.reset_index(inplace=True,drop=True)

mod = df

- join model and observed dataframe

In [407]:
df_out = pd.merge(mod,obs,on=['tod','NewINode'])

# Join with lat and lon
junctions = pd.read_csv('data/all_junctions.txt')

df_out = pd.merge(df_out, junctions[['PSRCjunctI','lat','lon']],left_on='NewINode',right_on='PSRCjunctI')
df_out.to_csv('sample_counts.csv')

In [409]:
df_out.head()

,model,tod,NewINode,observed,PSRCjunctI,lat,lon
0,464.327362,5to6,21763,928.166667,21763,47.931031,-122.260633
1,199.494034,5to6,21763,928.166667,21763,47.931031,-122.260633
2,902.610779,6to7,21763,1477.333333,21763,47.931031,-122.260633
3,392.331024,6to7,21763,1477.333333,21763,47.931031,-122.260633
4,1079.594116,7to8,21763,1514.133333,21763,47.931031,-122.260633


#  Network Summary

In [491]:
net_summary_df = pd.read_excel(r'J:\Projects\Soundcast\network_summary_detailed.xlsx', sheetname='Network Summary')
df = pd.DataFrame(net_summary_df.stack())
df['tod']= df.index.get_level_values(0)
df['fieldname'] = df.index.get_level_values(1)
df.rename(columns={0:'model_value'},inplace=True)
df.reset_index(inplace=True, drop=True)
# Drop the rows with TP_4k column headers
df.drop(df[df['fieldname'] == 'TP_4k'].index, inplace=True)
# Split the fields by vmt, vht, delay
df['facility_type'] = df.fieldname.apply(lambda row: row.split('_')[0])
df['metric'] = df.fieldname.apply(lambda row: row.split('_')[-1])

In [500]:
df.to_csv('network_summary.csv')

- add results from another run